# Imports

In [1]:
from utils.gui import interactive_segmentation_with_config

In [2]:
interactive_segmentation_with_config()

Selected processing mode: ramified
Loaded stack with shape (192, 1536, 1536)


Running initial ramified segmentation...
Initial memory usage: 3214.84 MB

Starting first pass microglia segmentation with anisotropy normalization degree = 1...
Spacing (z, y, x): [0.9947916666666666, 0.27621744791666664, 0.27621744791666664]
Target spacing: 0.9947916666666666
Full zoom factors (z, y, x): [1.0, 3.6014801895019684, 3.6014801895019684]
Interpolated zoom factors (degree=1, z, y, x): [1.0, 0.27766361256544503, 0.27766361256544503]
Expected downsampled shape: (192, 426, 426)
New spacing (adjusted for degree): (0.9947916666666666, 0.9947916666666666, 0.9947916666666666)
Downsampled volume shape: (192, 426, 426)
Smoothing volume...
Enhancing tubular structures with ultra-low memory usage...
Processing 192 slices with 1 scales
Processing scale [1.0052356020942408, 1.0052356020942408, 1.0052356020942408] (1/1)...


Scale [1.0052356020942408, 1.0052356020942408, 1.0052356020942408]: 100%|██████████| 192/192 [03:13<00:00,  1.01s/it]


Applying Otsu thresholding with sensitivity 0.99...


100%|██████████| 4/4 [00:00<00:00, 211.63it/s]


...

100%|██████████| 86/86 [00:40<00:00,  2.10it/s]

Finalizing results...


Current memory usage: 9424.47 MB
Converting result to numpy array...


100%|██████████| 4/4 [00:00<00:00, 53.18it/s]


Cell splitting complete:
  Original cells: 86
  Cells evaluated: 86
  Cells split: 1
  New cells created: 132
  Final cell count: 86
Final memory usage: 9424.57 MB
Calculating ramified features...
Found 86 unique masks. Starting processing with 23 parallel jobs.
Processing in batches of 1 masks.
Extracting surface points for all masks...
Extracted surfaces for masks 1-1 of 86
Extracted surfaces for masks 2-2 of 86
Extracted surfaces for masks 3-3 of 86
Extracted surfaces for masks 4-4 of 86
Extracted surfaces for masks 5-5 of 86
Extracted surfaces for masks 6-6 of 86
Extracted surfaces for masks 7-7 of 86
Extracted surfaces for masks 8-8 of 86
Extracted surfaces for masks 9-9 of 86
Extracted surfaces for masks 10-10 of 86
Extracted surfaces for masks 11-11 of 86
Extracted surfaces for masks 12-12 of 86
Extracted surfaces for masks 13-13 of 86
Extracted surfaces for masks 14-14 of 86
Extracted surfaces for masks 15-15 of 86
Extracted surfaces for masks 16-16 of 86
Extracted surfaces for

In [4]:
import os
import pandas as pd

def organize_processing_dir(drctry, mode):
    """
    A function to organize the directory with multiple samples for processing with 3D segmentation
    Inputs:
    - drctry: the directory containing tif files and a single csv file with the mapping of x,y,z dimensions to the corresponding tif files; csv file should have columns: 
    Filename, Width (um), Height (um), Slices, Depth (um)

    -mode: 'nuclear' or 'ramified', determines which config template to use
    """
    mode= 'ramified'
    #Make a list of all the tif files in the directory
    tif_files = [f for f in os.listdir(drctry) if f.endswith('.tif')]
    #Make a list of all the csv files in the directory
    csv_files = [f for f in os.listdir(drctry) if f.endswith('.csv')]
    #Check that there is only one csv file in the directory
    if len(csv_files) != 1:
        raise ValueError('There should be only one csv file in the directory')
    #Load the csv file
    csv_file = csv_files[0]
    csv_path = os.path.join(drctry, csv_file)
    df = pd.read_csv(csv_path)
    #Check that the csv file has the correct columns
    if not all([col in df.columns for col in ['Filename', 'Width (um)', 'Height (um)', 'Slices', 'Depth (um)']]):
        raise ValueError('The csv file should have columns: Filename, Width (um), Height (um), Slices, Depth (um)')
    #Check that the csv file has the correct number of rows
    if len(df) != len(tif_files):
        print(len(df), len(tif_files))
        print(tif_files)
        raise ValueError('The csv file should have the same number of rows as the number of tif files in the directory')
    #Create a new directory for each tif file
    root_names = [f.split('.')[0] for f in tif_files]
    for root_name in root_names:
        new_dir = os.path.join(drctry, root_name)
        os.makedirs(new_dir, exist_ok=True)
        #Move the tif file to the new directory
        tif_file = root_name + '.tif'
        tif_path = os.path.join(drctry, tif_file)
        new_tif_path = os.path.join(new_dir, tif_file)
        os.rename(tif_path, new_tif_path)
        
        #Copy the correct yaml config file to the new directory
        if mode == 'nuclear':
            config_file = 'nuclear_config.yaml'
        elif mode == 'ramified':
            config_file = 'ramified_config.yaml'
        #Template config is in the same directory as this script
        config_path = config_file
        new_config_path = os.path.join(new_dir, config_file)
        #Copy using python's functionality rather than shell comands to avoid issues due to spaces or special characters in the file path
        with open(config_path, 'r') as f:
            with open(new_config_path, 'w') as f2:
                f2.write(f.read())

        #Populate the yaml file with the correct dimensions
        """
        The yaml file has the following field:
        voxel_dimensions:
            x: 471.4
            y: 471.4
            z: 1

        We need to populate the x, y, and z fields with the correct values from the csv file
        """
        row = df[df['Filename'] == root_name]
        x = row['Width (um)'].values[0]
        y = row['Height (um)'].values[0]
        z = row['Depth (um)'].values[0]
        #Read the yaml file
        with open(new_config_path, 'r') as f:
            lines = f.readlines()
        #Identify the line number where the voxel_dimensions field starts
        for i, line in enumerate(lines):
            if 'voxel_dimensions' in line:
                start = i
                break
        #Populate the x, y, and z fields
        lines[start+1] = '  x: {}\n'.format(x)
        lines[start+2] = '  y: {}\n'.format(y)
        lines[start+3] = '  z: {}\n'.format(z)
        #Write the new yaml file
        with open(new_config_path, 'w') as f:
            f.writelines(lines)
    


In [5]:
drctry = '/home/kirill/Desktop/For_Kirill/Iba1_Morpho_BB_Blind (Copy)'
organize_processing_dir(drctry, 'ramified')